In [18]:
# !pip install huggingface-hub==0.2.1
# !pip install ruclip==0.0.2
# !pip install --upgrade huggingface-hub
# !pip uninstall ruclip
# !pip install huggingface-hub
# !pip uninstall transformers
!pip install --upgrade transformers tokenizers

^C


In [1]:
import matplotlib.pyplot as plt
import torch
import transformers
from transformers import CLIPProcessor, CLIPModel, CLIPTokenizer
from PIL import Image
import io
import base64
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\alex\Desktop\service\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class imgToImg:
    def __init__(self):
        self.device =  "cuda:0" if torch.cuda.is_available() else "cpu"
        self.model_ID = "openai/clip-vit-base-patch32"
        self.model, self.processor, self.tokenizer = get_model_info(model_ID, device)
    def get_model_info(model_ID, device):
        model = CLIPModel.from_pretrained(model_ID).to(device)
        processor = CLIPProcessor.from_pretrained(model_ID)
        tokenizer = CLIPTokenizer.from_pretrained(model_ID)
        return model, processor, tokenizer 
    def get_single_text_embedding(self,text): 
        inputs = self.tokenizer(text, return_tensors = "pt").to(device)
        text_embeddings = self.model.get_text_features(**inputs)
        embedding_as_np = text_embeddings.cpu().detach().numpy()
        return embedding_as_np
    def get_single_image_embedding(self,my_image):
        image = self.processor(
              text = None,
              images = my_image, 
              return_tensors="pt"
            )["pixel_values"].to(device)
        
        embedding = self.model.get_image_features(image)
        embedding_as_np = embedding.cpu().detach().numpy()
        return embedding_as_np
    def get_top_N_images(self,query, data, spots, top_K=100, search_criterion="text"):    
        # Text to image Search
        if(search_criterion.lower() == "text"):
          query_vect = self.get_single_text_embedding(query)

        # Image to image Search
        else: 
          query_vect = self.get_single_image_embedding(query)

        # Relevant columns
        revevant_cols = ["WikiData","Name", "City","Lon", "Lat", "russian_captions" ]

        # Run similarity Search
        data["cos_sim"] = data["img_embeddings"].apply(lambda x: cosine_similarity(query_vect, x))

        data["cos_sim"] = data["cos_sim"].apply(lambda x: x[0][0])

        filters = []
        n = 10
        most_similar = pd.DataFrame(data.sort_values(by='cos_sim', ascending=False))
        places = pd.DataFrame(columns= ["name", "image", "img_embeddings","pil", "cos_sim" ])
        # sights = pd.DataFrame(columns= ["WikiData","Name","Kind", "City","Rate","Lon", "Lat", "russian_captions" ])
        
        for index, row in most_similar.iterrows():
            if row["name"].lower() not in filters:
                places.loc[index] = row
                # sights.loc[index] = spots.where(spots["Name"] == row["name"]).iloc[0]
                filters.append(row["name"].lower())
            if len(places) == n:
                break
        return filters
    def plot_images_by_side(self,top_images, n_row, n_col):
        index_values = list(top_images.index.values)
        list_images = [top_images.iloc[idx].pil for idx in index_values] 
        similarity_score = [top_images.iloc[idx].cos_sim for idx in index_values] 

          
        _, axs = plt.subplots(n_row, n_col, figsize=(15, 15))
        axs = axs.flatten()
        for img, ax,  sim_score in zip(list_images, axs,  similarity_score):
                ax.imshow(img)
                sim_score = 100*float("{:.2f}".format(sim_score))
                ax.title.set_text(f"Similarity: {sim_score}%")
        plt.show() 
    def get_total_embds(self,df, df_places, img_column, captions_column):
        temp_lst = []
    
        for index, row in df.iterrows():
            temp_lst.append(self.get_single_image_embedding(row["pil"]))
        # df["img_embeddings"] = temp_lst

        temp_lst = np.array(temp_lst)
        
        for i in range(len(temp_lst.shape[1])):
            df[f"embedding_{i}"] = temp_lst[:, i]
    
#    temp_lst = []
#    for index, row  in df_places.iterrows():
#        temp_vec = 0
#        for j in row[captions_column].split(" "):
#                temp_vec += get_single_text_embedding(j)
#        temp_lst.append(temp_vec)
#    df_places["captions_embds"] = temp_lst
#    temp_lst = []
#    for index, row in df.iterrows():
#        temp_lst.append(df_places[df_places["Name"] == row["name"]]["captions_embds"])
#      
#    df["captions_embds"] = temp_lst
#    df["total"] = df["captions_embds"] + df["img_embeddings"]
        return df
    def dataToJpg(self, data, source):
        for i in data:
            temp = []
            for j in range(len(i["image"])):
                temp.append(Image.open(io.BytesIO(base64.b64decode(list(i["image"])[j]))))
            i["pil"] = temp
     
     
    

In [3]:
def get_model_info(model_ID, device):
    
     model = CLIPModel.from_pretrained(model_ID).to(device)
   
    
     processor = CLIPProcessor.from_pretrained(model_ID)
    
    
     tokenizer = CLIPTokenizer.from_pretrained(model_ID)
    
     return model, processor, tokenizer
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model_ID = "openai/clip-vit-base-patch32"
model, processor, tokenizer = get_model_info(model_ID, device)

In [4]:
def get_single_text_embeddingg(text): 
    inputs = tokenizer(text, return_tensors = "pt").to(device)
    text_embeddings = model.get_text_features(**inputs)
    embedding_as_np = text_embeddings.cpu().detach().numpy()

    return embedding_as_np

In [5]:
def get_single_image_embeddingg(my_image):

  image = processor(
      text = None,
      images = my_image, 
      return_tensors="pt"
  )["pixel_values"].to(device)

  embedding = model.get_image_features(image)

  
  embedding_as_np = embedding.cpu().detach().numpy()

  return embedding_as_np

In [6]:
def get_top_N_imagess(query, data, top_K=100, search_criterion="text"):
    # Text to image Search
    if(search_criterion.lower() == "text"):
      query_vect = get_single_text_embedding(query)

    # Image to image Search
    else: 
      query_vect = get_single_image_embedding(query )

    # Relevant columns
    revevant_cols = ["image", "cos_sim"]

    # Run similarity Search
    data["cos_sim"] = data["img_embeddings"].apply(lambda x: cosine_similarity(query_vect, x))

    data["cos_sim"] = data["cos_sim"].apply(lambda x: x[0][0])
    
    filters = []
    n = 10
    most_similar = pd.DataFrame(data.sort_values(by='cos_sim', ascending=False))
    places = pd.DataFrame(columns= ["name", "image", "img_embeddings", "cos_sim" ])
    
    for index, row in most_similar.iterrows():
        if row["name"].lower() not in filters:
            places.loc[index] = row
            filters.append(row["name"].lower())
        if len(places) == n:
            break
    
    return places[revevant_cols].reset_index()

In [7]:
def plot_images_by_sidee(top_images):

  index_values = list(top_images.index.values)
  print(len(index_values))
  list_images = [top_images.iloc[idx].image for idx in index_values] 
  
  similarity_score = [top_images.iloc[idx].cos_sim for idx in index_values] 

  n_row = 5
  n_col = 2
  _, axs = plt.subplots(n_row, n_col, figsize=(15, 15))
  axs = axs.flatten()
  for img, ax,  sim_score in zip(list_images, axs,  similarity_score):
      ax.imshow(img)
      sim_score = 100*float("{:.2f}".format(sim_score))
      ax.title.set_text(f"Similarity: {sim_score}%")
  plt.show()

In [8]:
def get_total_embdss(df, df_places, img_column, captions_column):
    temp_lst = []
    
    for index, row in df.iterrows():
        
        temp_lst.append(get_single_image_embedding(row["image"]))
    df["img_embeddings"] = temp_lst
    
#    temp_lst = []
#    for index, row  in df_places.iterrows():
#        temp_vec = 0
#        for j in row[captions_column].split(" "):
#                temp_vec += get_single_text_embedding(j)
#        temp_lst.append(temp_vec)
#    df_places["captions_embds"] = temp_lst
#    temp_lst = []
#    for index, row in df.iterrows():
#        temp_lst.append(df_places[df_places["Name"] == row["name"]]["captions_embds"])
#      
#    df["captions_embds"] = temp_lst
#    df["total"] = df["captions_embds"] + df["img_embeddings"]
    return df

In [9]:
EKB_img = pd.read_csv("data/EKB_images.csv")
# NN_img = pd.read_csv("data/NN_images.csv")
# VLD_img = pd.read_csv("data/Vladimir_images.csv")
# YAR_img = pd.read_csv("data/Yaroslavl_images.csv")
EKB_places = pd.read_csv("data/EKB_places_pure.csv")
# NN_places = pd.read_csv("data/NN_places_pure.csv")
# VLD_places = pd.read_csv("data/VLAD_places_pure.csv")
# YAR_places = pd.read_csv("data/YAROSLAVL_places_pure.csv")

In [10]:
data_img = [EKB_img] # NN_img, VLD_img, YAR_img
data_places = [EKB_places] # NN_places, VLD_places, YAR_places

In [11]:
model = imgToImg()

In [12]:
model.dataToJpg(data_img, "image")


In [13]:
EKB_img.head()

,name,image,pil
0,Динамо,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...,<PIL.JpegImagePlugin.JpegImageFile image mode=...
1,Динамо,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...,<PIL.JpegImagePlugin.JpegImageFile image mode=...
2,Динамо,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...,<PIL.JpegImagePlugin.JpegImageFile image mode=...
3,Динамо,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...,<PIL.JpegImagePlugin.JpegImageFile image mode=...
4,Динамо,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...,<PIL.JpegImagePlugin.JpegImageFile image mode=...


In [14]:
EKB_withEmbds = model.get_total_embds(EKB_img, EKB_places, "pil", "Kind")

ValueError: Expected a 1D array, got an array with shape (2572, 512)

In [ ]:
EKB_withEmbds.head()

In [41]:
# Save data
EKB_withEmbds.to_csv("data/EKB_images_emb.csv")
# NN_withEmbds.to_csv("data/NN_images_emb.csv")
# VLD_withEmbds.to_csv("data/Vladimir_images_emb.csv")
# YAR_withEmbds.to_csv("data/Yaroslavl_images_emb.csv")

In [37]:
EKB_withEmbds.head()

,name,image
0,Динамо,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
1,Динамо,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
2,Динамо,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
3,Динамо,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
4,Динамо,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...


In [27]:
EKB_img['img_embeddings'].iloc[0].shape

(1, 512)

In [38]:
query_image = EKB_img.iloc[1920].pil


<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=427x320 at 0x21F32D13890>


"WikiData","Name","Kind", "City","Rate","Lon", "Lat", "russian_captions"

In [78]:

top_images  = model.get_top_N_images(query_image, EKB_img, EKB_places, search_criterion="image" )
top_images
#model.plot_images_by_side(top_images, 5, 5)

['улица чернышевского',
 'улица добролюбова',
 'дом печати (екатеринбург)',
 'дом связи (екатеринбург)',
 'дом промышленности',
 'белая башня',
 'театр музыкальной комедии',
 '№32 дом обороны',
 'дом обороны',
 '№34 дом контор']

In [92]:
def getData(names, places):
    result_df = pd.DataFrame(columns= ["WikiData","Name","Kind", "City","Rate","Lon", "Lat", "russian_captions" ])
    for name in names:
        # Find entries with the given name in the 'Name' column of the DataFrame (case-insensitive)
        entries = places[places["Name"].str.lower() == name.lower()]
        
        # If entries are found, append them to the result DataFrame
        if not entries.empty:
            result_df = pd.concat([result_df, entries])
        else:
            print(name)
    
    return result_df

data  = getData(top_images,EKB_places )

№32 дом обороны
№34 дом контор


C:\Users\alex\AppData\Local\Temp\ipykernel_22128\306915455.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, entries])


In [94]:
EKB_places[EKB_places['Name'] == '№32 Lом Обороны']

,WikiData,Name,Kind,City,Rate,Lon,Lat,russian_captions
0,Q106150540,"Здание бывшей гостиницы ""Мадрид""","architecture,historic_architecture,accomodatio...",Екатеринбург,3h,60.588634,56.886154,архитектура историческая архитектура места про...
1,Q107506739,Серго Орджоникидзе,"historic,monuments_and_memorials,interesting_p...",Екатеринбург,3h,60.592449,56.885479,историческое монументы и мемориалы интересные ...
2,Q107538882,Н. И. Кузнецову,"historic,monuments_and_memorials,interesting_p...",Екатеринбург,3h,60.577671,56.890465,историческое монументы и мемориалы интересные ...
3,Q107573037,Малышеву,"historic,monuments_and_memorials,urban_environ...",Екатеринбург,3h,60.604321,56.834213,историческое монументы и мемориалы городская с...
4,Q107583229,Комсомолу Урала,"historic,monuments_and_memorials,urban_environ...",Екатеринбург,3h,60.610554,56.844635,историческое монументы и мемориалы городская с...


In [90]:
top_images

['улица чернышевского',
 'улица добролюбова',
 'дом печати (екатеринбург)',
 'дом связи (екатеринбург)',
 'дом промышленности',
 'белая башня',
 'театр музыкальной комедии',
 '№32 дом обороны',
 'дом обороны',
 '№34 дом контор']